In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [41]:
batch_sz = 4

def train_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
# #     inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
#     inp = [numpy.dstack([scene['p_in']]) for scene in batch]
# #     out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
#     out = [numpy.dstack([scene['p_out']]) for scene in batch]
#     inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:], scene['v_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
#     out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
#     out = [numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:], scene['v_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    out = [numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
#     inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:], scene['v_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=train_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = False, collate_fn=test_collate, num_workers=0)

### Visualize the batch of sequences

In [4]:
import matplotlib.pyplot as plt
import random

agent_id = 0

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """Encoder Network."""
    def __init__(self,
                 input_size: int = 2,
                 embedding_size: int = 8,
                 hidden_size: int = 16):
        """Initialize the encoder network.

        Args:
            input_size: number of features in the input
            embedding_size: Embedding size
            hidden_size: Hidden size of LSTM

        """
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(input_size, embedding_size)
        self.lstm1 = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x: torch.FloatTensor, hidden: Any) -> Any:
        """Run forward propagation.

        Args:
            x: input to the network
            hidden: initial hidden state
        Returns:
            hidden: final hidden 

        """
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm1(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        """Initialize the decoder network.

        Args:
            embedding_size: Embedding size
            hidden_size: Hidden size of LSTM
            output_size: number of features in the output

        """
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm1 = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        """Run forward propagation.

        Args:
            x: input to the network
            hidden: initial hidden state
        Returns:
            output: output from lstm
            hidden: final hidden state

        """
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm1(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [37]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(train_loader):
        
        inp, out = sample_batch
        # preprocessing more ????
        inp = inp[:,0,:,:]
        out = out[:,0,:,:]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        # add in seq len?????
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        # initialize losses
        loss = 0
        
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            
#             print(encoder_input.shape)
#             print(encoder_hidde.shape)
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += criterion(decoder_output[:, :2], target[:, di, :2])

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
#         output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        iterator.set_postfix(loss=(loss.item()*_input.size(0) / (counter * train_loader.batch_size)))

In [38]:
device = "cuda"
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 1

for epoch in range(1, num_epoch + 1):
        train(encoder, decoder, device, val_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

In [39]:
def predict(model, device, test_loader):
    model.eval()
    counter = 0
    all_outs = numpy.zeros((1,60))
    idx_out = []
#     print(all_outs.shape)
    with torch.no_grad():
        for data, idx in test_loader:
            data= data[:,0,:,:]
            data= data.to(device)
            output = model(data)
            output = output[:,:,0:2].reshape(-1,60)
            arr = output.data.cpu().numpy()
#             print(arr.shape)
            all_outs = numpy.append(all_outs, arr, axis=0)
#             print(all_outs)
#             print(idx)
#             print('')
            idx_out.append(int(idx[0][0][0][0]))
            idx_out.append(int(idx[1][0][0][0]))
            idx_out.append(int(idx[2][0][0][0]))
            idx_out.append(int(idx[3][0][0][0]))
#     print(all_outs.shape)
#     print(idx_out[0])
#     print(len(idx_out))
    all_outs = all_outs[1:,:]
#     all_outs = numpy.insert(all_outs, 0, idx_out, axis=1)
#     for i in range(len(all_outs)):
#         print(all_outs[i][0])
#         all_outs[i][0] = idx_out[i]
#     print(all_outs.shape)\
    print(all_outs)
    return all_outs, idx_out

In [89]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.

    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
#     args = parse_arguments()
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
        _input = _input[:,0,:,:]
        
        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 2)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get absolute trajectory
#         abs_helpers = {}
#         abs_helpers["REFERENCE"] = np.array(helpers_dict["DELTA_REFERENCE"])
#         abs_helpers["TRANSLATION"] = np.array(helpers_dict["TRANSLATION"])
#         abs_helpers["ROTATION"] = np.array(helpers_dict["ROTATION"])
#         abs_inputs, abs_outputs = baseline_utils.get_abs_traj(
#             _input.clone().cpu().numpy(),
#             decoder_outputs.detach().clone().cpu().numpy(),
#             args,
#             abs_helpers,
#         )

        for i in range(30):
#             print(decoder_outputs.shape)
            for j in range(4):
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:])
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:]]
                
                
                
#             print(decoder_outputs[1,i,:])
#             print(decoder_outputs[2,i,:])
            
#             print(idx[1])
#             print(forecasted_trajectories[str(idx)])
#             idx_out.append(int(idx[0][0][0][0]))
#             idx_out.append(int(idx[1][0][0][0]))
#             idx_out.append(int(idx[2][0][0][0]))
#             idx_out.append(int(idx[3][0][0][0]))
#             forecasted_trajectories[idx] = decoder_outputs[i]

    return(forecasted_trajectories)

In [88]:
output = infer_absolute(test_loader, encoder, decoder)
print(output)

torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size(

torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size(

torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size(

torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size(

torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size(

...

torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size(

torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size(

torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size(

torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size([4, 30, 2])
torch.Size(

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [91]:
output

{10002: [tensor([603.7028, 667.0202], device='cuda:0', grad_fn=<SliceBackward>),
  tensor([734.8952, 811.7399], device='cuda:0', grad_fn=<SliceBackward>),
  tensor([758.7953, 838.0922], device='cuda:0', grad_fn=<SliceBackward>),
  tensor([762.2280, 841.8769], device='cuda:0', grad_fn=<SliceBackward>),
  tensor([762.6966, 842.3934], device='cuda:0', grad_fn=<SliceBackward>),
  tensor([762.7601, 842.4634], device='cuda:0', grad_fn=<SliceBackward>),
  tensor([762.7687, 842.4729], device='cuda:0', grad_fn=<SliceBackward>),
  tensor([762.7698, 842.4741], device='cuda:0', grad_fn=<SliceBackward>),
  tensor([762.7700, 842.4742], device='cuda:0', grad_fn=<SliceBackward>),
  tensor([762.7700, 842.4744], device='cuda:0', grad_fn=<SliceBackward>),
  tensor([762.7700, 842.4744], device='cuda:0', grad_fn=<SliceBackward>),
  tensor([762.7700, 842.4744], device='cuda:0', grad_fn=<SliceBackward>),
  tensor([762.7700, 842.4744], device='cuda:0', grad_fn=<SliceBackward>),
  tensor([762.7700, 842.4744], 

In [ ]:
HEAD_STR = "v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60"

numpy.savetxt("outputs.csv", output, delimiter=",", header = HEAD_STR)

In [ ]:
output.shape

In [ ]:
import pandas as pd
df = pd.read_csv("outputs.csv")
df = df.rename(columns={'# v1': 'v1'})
df

In [ ]:
df.insert(0, 'ID', idx_out)
df.to_csv("outputs.csv", index=False)

In [ ]:
df

In [ ]:
# numpy.loadtxt("oututs.csv", delimiter=',').shape

## TRYING SKLEARN.ENSEMBLE